In [1]:
!pip install ipywidgets

In [2]:
import pandas as pd

df = pd.read_excel('Comparative_IA.xlsx')
df.head()

,Original,Gemini,GPT - 4,Deepseek,Humano
0,Fecha apertura: 06/01/2021 08:08 Fecha: 06/01/...,# Historia Clínica\n\n## I. Información Genera...,# Consulta Médica - Teleconsulta\n\n## Informa...,**Fecha apertura:** 06/01/2021 08:08 \n**Fech...,-
1,Fecha apertura: 18/03/2021 10:28 Fecha: 18/03/...,# Historia Clínica\n\n---\n\n## I. Información...,# Seguimiento de Enfermería - Hemodinamia Adul...,**Fecha apertura:** 18/03/2021 10:28 \n**Fech...,-
2,Fecha: 18/03/2021 15:18 - Ubicación: HEMODINAM...,# Historia Clínica\n\n---\n\n## I. Información...,# Resumen de Atención - Hemodinamia Adultos\n\...,**Fecha:** 18/03/2021 15:18 \n**Ubicación:** ...,-


In [5]:
import pandas as pd
from rouge_score import rouge_scorer
import bert_score

df = pd.read_excel("Comparative_IA.xlsx")

# Using columns in a program
df.columns = df.columns.str.strip()

# Define parameters
referencias = df["Original"]
modelos = ["Gemini", "GPT - 4", "Deepseek", "Humano"]

# ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to apply ROUGE
def compute_rouge_scores(referencias, predicciones):
    r1, r2, rl = [], [], []
    for ref, pred in zip(referencias, predicciones):
        scores = scorer.score(str(ref), str(pred))
        r1.append(scores['rouge1'].fmeasure)
        r2.append(scores['rouge2'].fmeasure)
        rl.append(scores['rougeL'].fmeasure)
    return r1, r2, rl

# Calculate metrics for each model
for modelo in modelos:
    preds = df[modelo]
    
    # ROUGE
    r1, r2, rl = compute_rouge_scores(referencias, preds)
    df[f"{modelo}_rouge1"] = r1
    df[f"{modelo}_rouge2"] = r2
    df[f"{modelo}_rougeL"] = rl

    # BERTScore
    P, R, F1 = bert_score.score(preds.astype(str).tolist(), referencias.astype(str).tolist(), lang="es", verbose=False)
    df[f"{modelo}_bertscore_F1"] = F1


In [6]:
df = pd.read_excel('Comparative_IA.xlsx')

# Check if metrics have been calculated
metric_columns = [col for col in df.columns if any(metric in col for metric in ['rouge1', 'rouge2', 'rougeL', 'bertscore'])]

if not metric_columns:
    print("❌ No metrics found! Please run the notebook first to calculate ROUGE and BERTScore metrics.")
else:
    print("✅ Metrics found!")
    print(f"📊 Found {len(metric_columns)} metric columns")
    
    # Define models
    modelos = ["Gemini", "GPT - 4", "Deepseek", "Humano"]
    
    # Display sample metrics
    print(f"\n📈 Sample metrics (first 3 rows):")
    print(df[metric_columns].head(3))
    
    # Summary statistics
    print("\n" + "="*60)
    print("📊 SUMMARY STATISTICS BY MODEL")
    print("="*60)
    
    for modelo in modelos:
        print(f"\n🤖 {modelo.upper()}:")
        print("-" * 40)
        
        # ROUGE metrics
        rouge1_mean = df[f"{modelo}_rouge1"].mean()
        rouge2_mean = df[f"{modelo}_rouge2"].mean()
        rougeL_mean = df[f"{modelo}_rougeL"].mean()
        
        # BERTScore
        bertscore_mean = df[f"{modelo}_bertscore_F1"].mean()
        
        print(f"ROUGE-1:  {rouge1_mean:.4f}")
        print(f"ROUGE-2:  {rouge2_mean:.4f}")
        print(f"ROUGE-L:  {rougeL_mean:.4f}")
        print(f"BERTScore: {bertscore_mean:.4f}")
    
    # Create comparison table
    print("\n" + "="*60)
    print("🏆 MODEL COMPARISON TABLE")
    print("="*60)
    
    comparison_data = []
    
    for modelo in modelos:
        comparison_data.append({
            'Model': modelo,
            'ROUGE-1': df[f"{modelo}_rouge1"].mean(),
            'ROUGE-2': df[f"{modelo}_rouge2"].mean(),
            'ROUGE-L': df[f"{modelo}_rougeL"].mean(),
            'BERTScore': df[f"{modelo}_bertscore_F1"].mean()
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.round(4)
    
    print(comparison_df.to_string(index=False))
    
    # Highlight best scores
    print("\n🥇 BEST PERFORMANCE BY METRIC:")
    for metric in ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BERTScore']:
        best_model = comparison_df.loc[comparison_df[metric].idxmax(), 'Model']
        best_score = comparison_df[metric].max()
        print(f"{metric}: {best_model} ({best_score:.4f})")
    
    # Save results
    print("\n💾 Saving results...")
    df.to_excel('Comparative_IA_with_metrics.xlsx', index=False)
    comparison_df.to_excel('Model_Comparison_Summary.xlsx', index=False)
    print("✅ Results saved!")

❌ No metrics found! Please run the notebook first to calculate ROUGE and BERTScore metrics.
